In [1]:
import ee
ee.Initialize()

In [2]:
composite = ee.Image('projects/crowtherlab/nina/treemap/composite_to_sample')
print(composite.bandNames().getInfo())

['SG_Coarse_fragments_005cm', 'SG_Silt_Content_005cm', 'SG_Soil_pH_H2O_005cm', 'CHELSA_bio12_1981_2010_V2_1', 'CHELSA_bio15_1981_2010_V2_1', 'CHELSA_bio1_1981_2010_V2_1', 'CHELSA_bio4_1981_2010_V2_1', 'CHELSA_gsl_1981_2010_V2_1', 'CHELSA_npp_1981_2010_V2_1', 'Abs_Lat', 'Pixel_Lat', 'Pixel_Long', 'Resolve_Biome', 'Resolve_Ecoregion']


In [3]:
mask_img = composite.select('Abs_Lat').add(10000)

In [64]:
time = ee.String('2011_2040_')
scenario = ee.String('ssp126')
chelsa_layers = ee.ImageCollection('users/crowtherlab/Chelsa_V2_1/CHELSA_' + time.getInfo() + scenario.getInfo())

In [65]:
def cov_names(climate_model):
    return ee.List([
        ee.String('CHELSA_bio12_'), ee.String('CHELSA_bio15_'), ee.String('CHELSA_bio1_'), 
        ee.String('CHELSA_bio4_'), ee.String('CHELSA_gsl_'), ee.String('CHELSA_npp_')
    ]).map(lambda s: ee.String(s).cat(time).cat(climate_model).cat(scenario))

In [66]:
cov_names('gfdl_esm4_').getInfo()

['CHELSA_bio12_2011_2040_gfdl_esm4_ssp126',
 'CHELSA_bio15_2011_2040_gfdl_esm4_ssp126',
 'CHELSA_bio1_2011_2040_gfdl_esm4_ssp126',
 'CHELSA_bio4_2011_2040_gfdl_esm4_ssp126',
 'CHELSA_gsl_2011_2040_gfdl_esm4_ssp126',
 'CHELSA_npp_2011_2040_gfdl_esm4_ssp126']

In [67]:
chelsa_covariates = ee.List(['gfdl_esm4_', 'ipsl_cm6a_lr_', 'mpi_esm1_2_hr_', 'mri_esm2_0_', 'ukesm1_0_ll_']).map(
    lambda model: chelsa_layers.filter(ee.Filter.inList('system:index', cov_names(model))))
print(chelsa_covariates.map(lambda ic: ee.ImageCollection(ic).aggregate_array('system:index').length()).getInfo())

[6, 6, 6, 6, 6]


In [68]:
def full_covariate_img(chelsa_cov):
    chelsa_cov_img = ee.Image(ee.ImageCollection(chelsa_cov).iterate(
        lambda img_to_add, final_img: ee.Image(final_img).addBands(
            img_to_add.select(['b1'], [img_to_add.get('system:index')]).updateMask(mask_img)
        ), ee.Image()
    )).select(ee.ImageCollection(chelsa_cov).aggregate_array('system:index'))
    
    full_cov_img = chelsa_cov_img.addBands(composite, [
        'SG_Coarse_fragments_005cm', 'SG_Silt_Content_005cm', 'SG_Soil_pH_H2O_005cm'
    ])
    full_cov_img = full_cov_img.select(full_cov_img.bandNames(), [
        'bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph'
    ])
    
    img_name = 'covariates_' + '_'.join(ee.ImageCollection(chelsa_cov).first().get('system:index').getInfo().split('_')[2:])
    print(img_name)
    export = ee.batch.Export.image.toAsset(
        image = full_cov_img,
        description = 'treemap_' + img_name,
        assetId = 'projects/crowtherlab/nina/treemap/covariate_imgs/' + img_name,
        crs = 'EPSG:4326',
        crsTransform = '[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
        region = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False),
        maxPixels = int(1e13))
    export.start()
    
    return full_cov_img.bandNames().getInfo()

In [69]:
for i in range(chelsa_covariates.length().getInfo()):
    print(i)
    print(full_covariate_img(chelsa_covariates.get(i)))

0
covariates_2011_2040_gfdl_esm4_ssp126
['bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph']
1
covariates_2011_2040_ipsl_cm6a_lr_ssp126
['bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph']
2
covariates_2011_2040_mpi_esm1_2_hr_ssp126
['bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph']
3
covariates_2011_2040_mri_esm2_0_ssp126
['bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph']
4
covariates_2011_2040_ukesm1_0_ll_ssp126
['bio12', 'bio15', 'bio1', 'bio4', 'gsl', 'npp', 'coarse_fragments', 'silt_content', 'soil_ph']
